In [1]:
import argparse
import logging
import os
import requests
import zipfile

In [2]:
model_dict = {
    '20170511-185253': '0B5MzpY9kBtDVOTVnU3NIaUdySFE'
}


def download_and_extract_model(model_name, data_dir):
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)

    file_id = model_dict[model_name]
    destination = os.path.join(data_dir, model_name + '.zip')
    if not os.path.exists(destination):
        print('Downloading model to %s' % destination)
        download_file_from_google_drive(file_id, destination)
        with zipfile.ZipFile(destination, 'r') as zip_ref:
            print('Extracting model to %s' % data_dir)
            zip_ref.extractall(data_dir)


def download_file_from_google_drive(file_id, destination):
    URL = "https://drive.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params={'id': file_id}, stream=True)
    token = get_confirm_token(response)

    if token:
        params = {'id': file_id, 'confirm': token}
        response = session.get(URL, params=params, stream=True)

    save_response_content(response, destination)


def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None


def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk:  # filter out keep-alive new chunks
                f.write(chunk)

In [3]:
logging.basicConfig(level=logging.DEBUG)
model_dir = 'etc/'
download_and_extract_model('20170511-185253', model_dir)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): drive.google.com:443


DEBUG:urllib3.connectionpool:https://drive.google.com:443 "GET /uc?export=download&id=0B5MzpY9kBtDVOTVnU3NIaUdySFE HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (2): drive.google.com:443
DEBUG:urllib3.connectionpool:https://drive.google.com:443 "GET /uc?export=download&id=0B5MzpY9kBtDVOTVnU3NIaUdySFE&confirm=LIlb HTTP/1.1" 302 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): doc-0k-as-docs.googleusercontent.com:443
DEBUG:urllib3.connectionpool:https://doc-0k-as-docs.googleusercontent.com:443 "GET /docs/securesc/dqliq3folau7nvqpj0q1bmamlglk38i0/oaipj63mvesrrfk5unbamc70b3g6359l/1633365225000/18056234690049221457/09356801339940943632Z/0B5MzpY9kBtDVOTVnU3NIaUdySFE?e=download HTTP/1.1" 302 0
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): docs.google.com:443
DEBUG:urllib3.connectionpool:https://docs.google.com:443 "GET /nonceSigner?nonce=kjkaaf6d1d7qs&continue=https://doc-0k-as-docs.googleusercontent.com/docs/securesc/dqliq

Extracting model to etc/
